In [ ]:
"""
ethereumetl export_blocks_and_transactions \
    --transactions-output /mnt/ssd_large/Ethereum/transactions_0_11761677.csv \
    --provider-uri file:///mnt/ssd_raid/Ethereum/geth/geth.ipc \
    --start-block 0 --end-block 11761677 \
    --batch-size 4096 --max-workers 48
    
ethereumetl export_receipts_and_logs \
    --transaction-hashes transactions_test_hashes.txt \
    --provider-uri file:///ssd_raid/eth/geth.ipc  \
    --receipts-output receipts_test.csv \
    --batch-size 4096 --max-workers 24
    
ethereumetl export_token_transfers \
    --output token_transfers.csv \
    --provider-uri file:///ssd_raid/eth/geth.ipc \
    --start-block 11358296 --end-block 11358396 \
    --batch-size 4096 --max-workers 24
    
ethereumetl export_geth_traces \
    --output traces_46147_11358295.csv \
    --provider-uri file:///ssd_raid/eth/geth.ipc \
    --start-block 11358290 --end-block 11358295 \
    --batch-size 4096 --max-workers 48
"""

In [1]:
import findspark
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as Types

In [2]:
# setup spark for processing transactions
findspark.init()
spark_conf = pyspark.SparkConf()\
    .setMaster('local[*]')\
    .set('spark.local.dir', '/mnt/scratch_a,/mnt/scratch_b,/mnt/scratch_c')\
    .set('spark.driver.memory', '32g')\
    .setAppName('eth')

spark_context = SparkContext(conf=spark_conf)
spark_session = SparkSession(spark_context)

In [3]:
# ethereum-etl extract_field is slow...
transactions = spark_session.read.csv('/mnt/ssd_large/Ethereum/data/transactions_0_11761677.csv', header=True)
transaction_hashes = transactions.select('hash')
transaction_hashes.write.csv('/mnt/ssd_large/Ethereum/data/transactions_0_11761677_hashes', header=False)

In [4]:
transactions = spark_session.read.csv('/ssd_raid/eth/etl/train_transactions.csv', header=True)\
    .withColumn('block_number', F.col('block_number').cast(Types.DoubleType()))
transactions.agg(F.min(F.col('block_number')), F.max(F.col('block_number'))).show()

+-----------------+-----------------+
|min(block_number)|max(block_number)|
+-----------------+-----------------+
|          46147.0|      1.1358295E7|
+-----------------+-----------------+

